# Polynomials with field coefficients <span class="tag" tag-name="page"><span class="smallcaps">page</span></span>

> **Remember to save a copy of the file so that you can edit it.**
> (Click "File", then "Save a copy to Drive")

## Installing the required packages

In [ ]:
# This installs the required packages.
# Don't delete this cell!
%pip install gmpy2
%pip install primefac
%pip install "git+https://github.com/t-huettemann/MTH4021-repository-experimental.git#subdirectory=modules/rings_and_fields"


## Setting up

In [ ]:
import rings_and_fields as rf


## Defining a field

We'll work with the field $\mathbb{F}_{251}$ for illustration. We need
to set up two Python objects: `F` representing the field
$\mathbb{F}_{251}$, and `P` representing the polynomial ring
$\mathbb{F}_{251}[x]$.

In [ ]:
F = rf.primefield(251)
P = rf.polynomialring_over_field(F)
print("Prime field of characteristic 251:", F)
print("Ring of polynomials with coefficients in F:", P)


## Defining polynomials, basic calculations

In [ ]:
f = P([27, 0, 12, 203, 33])
g = P([5, 127, 200])
print("f=", f)
print("g=", g)


In [ ]:
print("f * g =", f*g)
print("f - g =", f-g)
print("f^4=", f.power(4))
print("g*f = 0?", g.mult(f).is_zero())


You can multiply a polynomial by an integer, so you can write `3*f` or
`f*8` without problems.

In [ ]:
print("3*f=", 3*f)
print("f*8=", f*8)
print("120*f=", 120*f)


## Division with remainder

We can divide by *any* non-zero polynomial (as we're working with field
coefficients now!). The "quotient" is obtained with the method `div`,
the "remainder" with the method `mod`.

In [ ]:
print("Is g monic?", g.is_monic())
print("Division with remainder: f= g*q + r where")
q = f.div(g)
r = f.mod(g)
print("  q=", q)
print("  r=", r)


In [ ]:
print("Test this: g*q + r = f?")
print(g*q+r == f)


## Testing irreducibility

When working over a finite field, it is possible to check whether a
polynomial is irreducible or not (using Rabin's test).

Let's try out a small examples over $\mathbb{F}_{7}$ first. First the
necessary set-up:

In [ ]:
K = rf.primefield(7)
print("K=", K)
Q = rf.polynomialring_over_field(K)
print("Q=", Q)


We test two fixed polynomials, and also a randomly chosen polynomial of
degree at most 3.

In [ ]:
p = Q([1,2,3,4])
print("p=", p)
print("Is the polynomial", p, "irreducible?", p.is_irreducible())
q = Q([2,5,0,1])
print("Is the polynomial", q, "irreducible?", q.is_irreducible())
z = Q([K.random_element() for i in range(4)])
print("Is the polynomial", z, "irreducible?", z.is_irreducible())


Now let's try some higher-degree examples!

In [ ]:
p1 = Q([4,2,5,6,4,1,3,5,3,1,4,1])
print("p=", p)
print("Is the polynomial", p1, "irreducible?", p1.is_irreducible())
q1 = Q([4,2,5,5,4,1,3,5,3,1,4,1])
print("Is the polynomial", q1, "irreducible?", q1.is_irreducible())


## Another example

-   $K = \mathbb{F}_{19}$
-   $f = 2x^3 + 4x^2 + 9$
-   $g = 14x^2 + 12x + 13$

The resultant of $f$ and $g$ is $\mathrm{Res}_{3,2}(f,g) =
228,636 \equiv 9 \mod 19$, so $f$ and $g$ do not have a common
non-trivial factor.

Let's try to check this a bit. Is $228,636 \equiv 9 \mod 19$ actually
true?

In [ ]:
L = rf.primefield(19)
rr = L(228636)
print(rr, "= 228,636 in the field", L)


Looks good. Let's compute the remainder when dividing $f$ by $g$, to
make sure that $g$ isn't a factor of $f$:

In [ ]:
T = rf.polynomialring_over_field(L)
f = T([9,0,4,2])
print("f=", f)
g = T([13,12,14])
print("g=", g)
print("f % g=", f % g)


Since the remainder `f % g` is non-zero we know $g \nmid f$.

Let's change tack and check if $f$ has any roots; we'll do this the
naive way and simply loop over the elements of the coefficient
field $\mathbb{F}_{19}$, evaluate the polynomial and check whether the
result is zero.

In [ ]:
for t in L:
    if f.eval(t).is_zero():
        print("f has root", t)
    else:
        print(t, "is not a root of f")


``` example
0 is not a root of f
1 is not a root of f
2 is not a root of f
3 is not a root of f
4 is not a root of f
5 is not a root of f
6 is not a root of f
7 is not a root of f
8 is not a root of f
9 is not a root of f
10 is not a root of f
11 is not a root of f
12 is not a root of f
13 is not a root of f
14 is not a root of f
15 is not a root of f
16 is not a root of f
17 is not a root of f
18 is not a root of f
```

So $f$ has no roots. This actually means that $f$ is irreducible (since
$\deg(f) =3$), and thus can't possibly have a common factor with $g$
(such a factor, if non-trivial, would be of degree 1 or 2).

## How many polynomials have more than one root?

Just for fun, and totally unrelated to the above, let's find the roots
(if any) of a higher-degree polynomial:

In [ ]:
h = T([6,2,4,2,0,1,15,1])
print("h=", h)
roots = []
for t in L:
    if h.eval(t).is_zero():
        roots.append(str(t))
print("Roots of h:", roots)


It took me a while to find (by random experimentation) a polynomial with
more than one root. So how likely is it that a polynomial has more than
one root? Let's investigate this numerically, simply enumerating all
polynomials and counting how many of the have more than one root. It's
of course enough to deal with monic polynomials.

We start with monic polynomials of degree 2 as a proof-of-concept:

In [ ]:
num = 0
tot = 0
for a0 in L:
    for a1 in L:
        tot = tot + 1
        h = T([a0,a1,1])
        roots = []
        for t in L:
            if h.eval(t).is_zero():
                roots.append(str(t))
        if len(roots)>1:
            num = num+1
print(num, "of", tot, "monic polynomials of degree 2 have more than one root")


So about 47% of the tested polynomials have more than one root.

And since that worked well, let's try with degree 3:

In [ ]:
num = 0
tot = 0
for a0 in L:
    for a1 in L:
        for a2 in L:
            tot = tot + 1
            h = T([a0,a1,a2,1])
            roots = []
            for t in L:
                if h.eval(t).is_zero():
                    roots.append(str(t))
            if len(roots)>1:
                num = num+1
print(num, "of", tot, "monic polynomials of degree 3 have more than one root")


So about 19% of the tested polynomials have more than one root!

And how about degree 4?

In [ ]:
num = 0
tot = 0
for a0 in L:
    for a1 in L:
        for a2 in L:
            for a3 in L:
                tot = tot + 1
                h = T([a0,a1,a2,a3,1])
                roots = []
                for t in L:
                    if h.eval(t).is_zero():
                        roots.append(str(t))
                if len(roots)>1:
                    num = num+1
print(num, "of", tot, "monic polynomials of degree 4 have more than one root")


That's a whopping 28% of polynomials with at least one root.

**Suggestion**: Try polynomials of degree 5, or even degree 6. Computing
times will become quite noticeable but still bearable (I think). What's
your prediction?